In [ ]:
import numpy as np

from dmf import DirectMaxFlux, interpolate_fbenm

import sys
import os

import ase
from ase import Atoms
from ase.build import bulk, surface
from ase.io import read, write, Trajectory
from ase.optimize import LBFGS, BFGS, FIRE
from ase.constraints import FixAtoms, Hookean

import orb_models
from orb_models.forcefield import pretrained
from orb_models.forcefield.inference.calculator import ORBCalculator
from orb_models.forcefield.inference.d3_model import D3SumModel, AlchemiDFTD3

from atom_editor import AtomEditorUI

device = "cpu"  # or device="cuda"
orbff, atoms_adapter = pretrained.orb_v3_conservative_inf_omat(
  device=device,
  precision="float32-high",   # or "float32-highest" / "float64
)
orbff_d3 = D3SumModel(orbff, AlchemiDFTD3(functional="PBE", damping="BJ", compile=True))

calc = ORBCalculator(orbff_d3, atoms_adapter=atoms_adapter, device=device)
#calc = ORBCalculator(orbff, atoms_adapter=atoms_adapter, device=device)

In [ ]:
# 入力ファイル（ASEで読める形式ならOK: xyz, traj, POSCAR, cif ...）
react_path = "react.xyz"
prod_path  = "prod.xyz"

react = read(react_path)
prod  = read(prod_path)

ref_images = [react, prod]

In [ ]:
# correlated=True が推奨設定の一つ（FB-ENM）
mxflx_fbenm = interpolate_fbenm(ref_images, correlated=True)

# 初期経路（イメージ列）保存
write("dmf_ini.traj", mxflx_fbenm.images)

# DMF の内部表現（係数）も保存しておくと再実行が楽
coefs = mxflx_fbenm.coefs.copy()
np.save("dmf_ini_coefs.npy", coefs)

print("Initial images:", len(mxflx_fbenm.images))

In [ ]:
# nmove: 経路の自由度（まずは 3 が無難。複雑なら増やす）
mxflx = DirectMaxFlux(ref_images, coefs=coefs, nmove=3, update_teval=True)

# 例: EMT（デモ用） ※PBC系の本番はVASP/LAMMPS等へ置換
for img in mxflx.images:
    img.calc = calc

print("DMF images:", len(mxflx.images))

In [ ]:
# IPOPTログをファイルへ
mxflx.add_ipopt_options({
    "output_file": "dmf_ipopt.out",
    # 必要なら例：
    # "max_iter": 300,
    # "tol": 1e-6,
})

# tol: "middle" は実装例にある指定。必要に応じて変えてOK
mxflx.solve(tol="middle")

print("DMF optimization finished.")